In [1]:
import pandas as pd
import sqlite3
import json
import os
from pprint import pprint
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
PATH_TO_CORD = os.getenv("PATH_TO_CORD")

In [2]:
#delete tables
con = sqlite3.connect('../cord.db')
cur = con.cursor()

# Create table
cur.execute("DROP TABLE relevant_factors")

# Insert a row of data
cur.execute("DROP TABLE symptoms")

# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [3]:
def prepare_sliced_df_for_metadata(df, start, end) :
    df_slice = df[start:end]
    df_slice = df_slice.dropna()
    
    #get pdf_json_file location
    pdf_json_file = df_slice.iloc[:, 9]
    
    text = []
    for i, location in enumerate(pdf_json_file) :
        
        path = PATH_TO_CORD + '/' + str(location)
        
        if os.path.isfile(path) :

            f = open(path)

            # returns JSON object as a dictionary
            data = json.load(f)

            text_temp = ''

            count_text = 0
            for body in data['body_text'] :
                text_temp += body['text']
                count_text += 1
                if count_text > 3 :
                    break

            text.append(text_temp)

            # Closing file
            f.close()
            
        else : text.append('')
            
    df_slice['body_text'] = text
    return df_slice

In [4]:
def make_table(path_to_file, to_remove, table_name) :
    df = pd.read_csv(path_to_file)

    df.drop(to_remove,  axis=1, inplace=True, errors='ignore')
    
    df_slice = prepare_sliced_df_for_metadata(df, 0, 5000)
    
    conn = sqlite3.connect('../cord.db')
    df_slice.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    
    return df_slice

In [5]:
def append_to_table(path_to_file, to_remove, table_name) :
    df = pd.read_csv(path_to_file)

    df.drop(to_remove, axis=1, inplace=True, errors='ignore')
    df.drop_duplicates(subset=['Study'], inplace=True)
    conn = sqlite3.connect('../cord.db')
    df.to_sql(table_name, conn, if_exists='append', index=False)
    conn.close()
    
    return df

In [6]:
PATH_TO_CORD = '../../../../../../media/arnav/Arnav/ArnavCode/projects/cord-19'

In [ ]:
to_remove = ['sha', 'source_x', 'pmcid', 'mag_id', 'who_covidence_id', 'arxiv_id', 'pmc_json_files', 's2_id']
table_name = 'cord19'
path_to_file = PATH_TO_CORD + '/metadata.csv'
df = make_table(path_to_file, to_remove, table_name)

In [9]:
to_remove = ['Date', 'Study Type', 'Influential', 'Infuential', 'Influential (Y/N)', 'Date Published', 'Factors Described']
path_to_folder = '/Kaggle/target_tables/2_relevant_factors/'
for file_name in os.listdir(PATH_TO_CORD + path_to_folder) :
    path_to_file = PATH_TO_CORD + path_to_folder + file_name
    
    table_name = 'relevant_factors'
    df = append_to_table(path_to_file, to_remove, table_name)

In [8]:
to_remove = ['Range (Days)', 'Days', 'Date', 'Study Type', 'Aymptomatic', 'Manifestation', 'Frequency of Symptoms', 'Sample Size', 'Specific Sampled Viral load correlated to postive test', 'Age', 'Asymptomatic', 'Sample Obtained', 'Sample obtained', 'Asymptomatic Transmission', 'Characteristic Related to Question 2']
path_to_folder = '/Kaggle/target_tables/3_patient_descriptions/'
for file_name in os.listdir(PATH_TO_CORD + path_to_folder) :
    path_to_file = PATH_TO_CORD + path_to_folder + file_name
    
    table_name = 'symptoms'
    df = append_to_table(path_to_file, to_remove, table_name)

/home/arnav/anaconda3/envs/deep-learning/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [10]:
conn = sqlite3.connect('../cord.db')
pd.read_sql('SELECT * FROM symptoms LIMIT 0,30', conn)

,Unnamed: 0,Study,Study Link,Journal,Excerpt,Added On
0,0,Presymptomatic Transmission of SARS-CoV-2 — Si...,http://dx.doi.org/10.15585/mmwr.mm6914e1,CDC: Morbidity and Mortality Weekly Report,Among the 243 cases of COVID-19 reported in Si...,2020-05-24
1,1,Modes of contact and risk of transmission in C...,https://doi.org/10.1101/2020.03.24.20042606,medRxiv,"Only 1 (1/305, 0.33%) and 19 (19/576, 3.3%) cl...",2020-05-10
2,2,Temporal dynamics in viral shedding and transm...,https://doi.org/10.1038/s41591-020-0869-5,Nature Medicine,The estimated proportion of presymptomatic tra...,2020-04-08
3,3,Epidemiological characteristics of 2019 novel ...,https://doi.org/10.3760/cma.j.cn112150-2020022...,Chinese Journal of Preventative Medicine,Family secondary attack rate for subsequent ca...,2020-04-09
4,4,Epidemiological parameters of coronavirus dise...,https://doi.org/10.1101/2020.03.21.20040329,medRxiv,"In 102 (43.78%) infector-infectee pairs, trans...",2020-04-08
5,5,Transmission of corona virus disease 2019 duri...,https://doi.org/10.1101/2020.03.06.20031955,medRxiv,The majority of the secondary cases (73.0%) we...,2020-04-08
6,0,Temporal profiles of viral load in posterior o...,https://www.thelancet.com/journals/laninf/arti...,The Lancet Infectious disease,COVID-19 highest viral load correlated with in...,4/26/2020
7,4,Clinical and virological data of the first cas...,https://www.thelancet.com/journals/laninf/arti...,The Lancet Infectious disease,High viral loads in upper respiratory tract sa...,4/26/2020
8,8,Quantitative Detection and Viral Load Analysis...,https://academic.oup.com/cid/advance-article/d...,Oxford Academic,"viral load in the early,progressive stages wer...",4/26/2020
9,14,A Well Infant With Coronavirus Disease 2019 Wi...,https://academic.oup.com/cid/advance-article/d...,Oxford Academic,"Even in absence of clinical symptoms, high nas...",4/26/2020


In [19]:
conn.close()